In [ ]:
import re

def parse_traceback(tb_string):
    # Regular expressions to match different parts of the traceback
    file_line_func_pattern = r'File "(.+)", line (\d+), in (.+)'
    error_pattern = r'^(\w+(?:\.\w+)*): (.+)$'

    lines = tb_string.strip().split('\n')
    parsed_info = {
        'frames': [],
        'error_type': None,
        'error_message': None
    }

    for line in lines:
        # Check for file, line, and function information
        file_line_func_match = re.search(file_line_func_pattern, line)
        if file_line_func_match:
            parsed_info['frames'].append({
                'file': file_line_func_match.group(1),
                'line': int(file_line_func_match.group(2)),
                'function': file_line_func_match.group(3)
            })
        
        # Check for error type and message
        error_type_mapping = {
            "grpc._channel._InactiveRpcError": "Connection Timeout",
            "httpcore.ReadTimeout": "Connection Timeout",
            "pydantic.error_wrappers.ValidationError": "Wrong Tool Usage",
            "langgraph.errors.GraphRecursionError": "Backtracking Exceeded",
        }

        error_match = re.search(error_pattern, line)
        if error_match and not parsed_info['error_type']:
            parsed_info['error_type'] = error_match.group(1)
            parsed_info['error_type'] = parsed_info['error_type'].replace("anthropic.", "").replace("openai.", "")
            parsed_info['error_type'] = error_type_mapping.get(parsed_info['error_type'], parsed_info['error_type'])

            parsed_info['error_message'] = error_match.group(2)

    # If frames exist, get the last frame (where the error occurred)
    if parsed_info['frames']:
        last_frame = parsed_info['frames'][-1]
        parsed_info['error_file'] = last_frame['file']
        parsed_info['error_line'] = last_frame['line']
        parsed_info['error_function'] = last_frame['function']

    return parsed_info
